In [48]:
from typing import List

# this works, but it is too slow for adversarial test cases
class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        for row in board:
            print(row)
        
        #print()
        #print(words)

        letter_dict = {}
        for row in range(len(board)):
            for col in range(len(board[row])):
                letter = board[row][col]
                if letter in letter_dict:
                    letter_dict[letter].append((row, col))
                else:
                    letter_dict[letter] = [(row, col)]
        
        #print()
        #print(letter_dict)

        result = []
        path = []

        def dfs(word, i) -> bool:
            nonlocal path
            nonlocal result
            #print(" "*(i+1) + f"dfs called with word = {word}, i = {i}, path = {path}")
            
            if i == len(word):
                result.append(word)
                return True
            
            char = word[i]
            if i == 0:
                if char in letter_dict:
                    for r, c in letter_dict[char]:
                        path.append((r, c))
                        if dfs(word, i + 1):
                            path.pop()
                            return True
                        path.pop()
            else:
                if char in letter_dict:
                    for r, c in letter_dict[char]:
                        rm, cm = path[-1]
                        if ((r, c) not in path):
                            if (abs(r - rm) == 1 and c == cm) or (abs(c - cm) == 1 and r == rm):
                                path.append((r, c))
                                if dfs(word, i + 1):
                                    path.pop()
                                    return True
                                path.pop()

            return

        for word in words:
            if all (char in letter_dict for char in word):
                dfs(word, 0)
        
        return result


In [167]:
from typing import List

class TrieNode:
    def __init__(self, is_word = False):
        self.children = {}
        self.is_word = is_word
    
    def __repr__(self):
        if self.is_word:
            return f"{self.children}"
        else:
            return f"{self.children}"

class WordTrie:
    def __init__(self, board):
        self.root = TrieNode()
        self.board = board
        self.result = set()
        self.dirs = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    def add_word(self, word):
        #print(f"adding word {word}")
        cur_node = self.root
        for i, c in enumerate(word):
            if c in cur_node.children:
                if i == len(word) - 1:
                    node = cur_node.children[c]
                    node.is_word = True
                cur_node = cur_node.children[c]
            else:
                node = TrieNode()
                cur_node.children[c] = node
                cur_node = node
                if i == len(word) - 1:
                    node.is_word = True
        return
    
    def remove_word(self, word, end_node):
        if len(word) == 0:
            return
        
        #print(f" removing word {word}")
        #print(f" end_node: {end_node}")
        end_node.is_word = False
        if len(end_node.children) != 0:
            return
        
        stack = []
        cur_node = self.root
        for char in word:
            stack.append((cur_node, char))
            cur_node = cur_node.children[char]
            #print(f"stack[-1]: {stack[-1]}")

        #print(f"len(stack): {len(stack)}")
        #print(f" stack[-1][0]: {stack[-1][0]}")
        #print(f" stack[-1][1]: {stack[-1][1]}")
        #print(f" stack[-1][0].is_word: {stack[-1][0].is_word}")
        #print(f" len(stack[-1][0].children): {len(stack[-1][0].children)}")
        #print(f" stack[-1][0].children: {stack[-1][0].children}")
        #print(f" stack[-1][0].children[stack[-1][1]]: {stack[-1][0].children[stack[-1][1]]}")
        #print(f" len(stack[-1][0].children[stack[-1][1]].children): {len(stack[-1][0].children[stack[-1][1]].children)}")

        while stack and (not stack[-1][0].is_word) and (len(stack[-1][0].children[stack[-1][1]].children) == 0):
            parent, char = stack.pop()
            parent.children.pop(char)
        
    def find_words_at(self, i, j):
        visited = set()
        m, n = len(self.board), len(self.board[0])

        def dfs(r, c, cur_word, cur_node):
            if self.board[r][c] in cur_node.children:
                cur_word += self.board[r][c]
                node = cur_node.children[self.board[r][c]]
                if node.is_word:
                    self.result.add(cur_word)
                    self.remove_word(cur_word, node)
                for dr, dc in self.dirs:
                    if (0 <= r+dr < m) and (0<=c+dc<n):
                        if (r+dr, c+dc) not in visited:
                            visited.add((r+dr, c+dc))
                            dfs(r+dr, c+dc, cur_word, node)
                            visited.remove((r+dr, c+dc))
            return
        visited.add((i,j))
        dfs(i, j, '', self.root)
        return
    
    def find_words(self):    
        m, n = len(self.board), len(self.board[0])
        for i in range(m):
            for j in range(n):
                self.find_words_at(i, j)
                #print(f"({i}, {j})")
                #print(" " + str(self.root))
        return list(self.result)

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        #for row in board:
        #    print(row)
        #print()

        trie = WordTrie(board)
        for word in words:
            trie.add_word(word)

        #print(trie.root)
        #print()

        result = trie.find_words()
        return result

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.end = False
    
    def __repr__(self):
        return f"{self.children}"
    
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def add(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.end = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.end

    def remove(self, word):
        def _remove(node, word, depth):
            if depth == len(word):
                if node.end:
                    node.end = False
                return len(node.children) == 0
            
            char = word[depth]
            if char not in node.children:
                return False
            
            delete_node = _remove(node.children[char], word, depth + 1)

            if delete_node:
                del node.children[char]
                return len(node.children) == 0
            
            return False

        if not self.search(word):
            return
        _remove(self.root, word, 0)





In [170]:
# test 1
board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
words = ["oath","pea","eat","rain", "rat"]
expected = ["eat","oath"]
output = Solution().findWords(board, words)
print("output = ", output, "expected = ", expected)
print([word in output for word in expected], [word in expected for word in output])
print()

# test 2
board = [["a","b"],["c","d"]]
words = ["abcb"]
expected = []
output = Solution().findWords(board, words)
print("output = ", output, "expected = ", expected)
print([word in output for word in expected], [word in expected for word in output])
print()

# test 3
board = [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]]
words = ["oa","oaa"]
expected = ["oa","oaa"]
output = Solution().findWords(board, words)
print("output = ", output, "expected = ", expected)
print([word in output for word in expected], [word in expected for word in output])
print()

output =  ['oath', 'eat'] expected =  ['eat', 'oath']
[True, True] [True, True]

output =  [] expected =  []
[] []

output =  ['oaa', 'oa'] expected =  ['oa', 'oaa']
[True, True] [True, True]

